### References Langchain
https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

### References Chroma db  
https://docs.trychroma.com/guides  
https://github.com/neo-con/chromadb-tutorial  
https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/  


In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import os
import sys
sys.path.append('../..')

from py3810.myUtils import pickle_dump, pickle_load
path_lumen_docs = '..\langchain\docs\lumen\\docs\\'
path_chatbot = '..\langchain\chatbot\\'

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env\my_api_key.env')) # read local .env file

os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]
os.environ["SECRET_KEY"]


'my_secret_key'

In [3]:
import textwrap

def print_wrapped(text, width=80):
  """
  Prints a long string to the console, wrapped to fit within a specified width.

  Args:
      text: The long string to be wrapped.
      width: The desired width for each line (default: 80 columns).
  """
  wrapped_text = textwrap.wrap(text, width=width)
  for line in wrapped_text:
    print(line)

# Example usage
long_string = "This is a very long string that needs to be wrapped to fit within 80 columns. It can contain spaces, punctuation, and even newlines."
print_wrapped(long_string)

This is a very long string that needs to be wrapped to fit within 80 columns. It
can contain spaces, punctuation, and even newlines.


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
persist_directory = path_chatbot + "./chroma_openai_ef"
embedding_function = OpenAIEmbeddings()
collection_name = 'lumen_docs_combined'

#### Only run cell below to add new data


In [ ]:
# # load data
# docs = pickle_load(filename_pickle='lumen_docs_combined', path_pickle_dump=path_chatbot)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# # save to disk
# db0 = Chroma.from_documents(
#   documents=splits,
#   embedding=embedding_function,
#   collection_name=collection_name,
#   persist_directory=persist_directory
#   )
# db0_ans = db0.similarity_search(query)
# # print(docs[0].page_content)
# print(f'db0_ans:{db0_ans}')

In [6]:
my_query = 'where is lumen optometric'

# load from disk
vectorstore = Chroma(
  embedding_function=embedding_function,
  collection_name=collection_name,
  persist_directory=persist_directory
  )
ans = vectorstore.similarity_search(my_query)
print(f'ans:{ans}')

ans:[Document(page_content='testtest Cutting-Edge Eye Care In The San Gabriel Valley\nWhere the Latest Research & Technology Converge With a Highly Personalized Education\nLumen Optometric is committed to providing clear vision and healthy eyes to the residents of Sierra Madre, Arcadia, Pasadena, and all surrounding communities. We’re shining a new light on patient care, so if you want cutting-edge service and holistic eye care that can’t be beat, the local experts at Lumen Optometric are here to help.\nPatients Are at the Heart of Everything We Do\nOur primary tenet is to provide excellent patient care that balances clarity, comfort and optimal health over your lifetime. Lumen Optometric has knowledgeable, caring eye specialists who are more than happy to share their time and expertise with you. Our areas of concentration include:\nComprehensive eye exams\nOrthokeratology (Ortho-K)\nNeurolens therapy\nScleral lenses for Keratoconus or High Prescriptions', metadata={'description': 'Lum

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
queries = [
  "What is lumen optometric's phone number?",
  "What is lumen optometric's address?",
  "What is lumen optometric's location?",
  "do you take vision insurance",
  "what type of insurance do you take",
  "what are the names of the insurance that you take",
  "What is Ortho-k",
  "What does research say about Ortho-k",
  "What equipment do they have?",
  "What equipment do you use for othro-k?",
  ]

In [10]:
# for query in queries[0:2]:
for query in queries[0:3]:
  print(f'query: {query}')
  print_wrapped(f'answr: {rag_chain.invoke(query)}')
  print('')
  print_wrapped(f'relevant docs: {retriever.get_relevant_documents(query)}')
  print(f'{"="*5}\n')

query: What is lumen optometric's phone number?
answr: You can reach Lumen Optometric at (626) 921-0199.

relevant docs: [Document(page_content='(626)
921-0199\ninfo@lumenoptometric.comServicesComprehensive Eye Exams\nContact Lens
Exams\nOrthokeratology\nNeurolens Therapy\nScleral Lenses / Keratoconus\nQuick
LinksHome\nAbout Us\nSitemap\nHours Of OperationTuesday9:45 am - 5:30
pmWednesday9:45 am - 5:30 pmThursday9:45 am - 1:30 pmFriday9:45 am - 5:30
pmSaturday9:45 am - 5:30 pmCopyright © 2020 Lumen Optometric. All Rights
Reserved.X[contact-form-7 404 "Not Found"]Top', metadata={'description': 'Lumen
Optometric is the only eye clinic in Sierra Madre, CA to offer 100% doctor-
designed Ortho-K lenses. We bring clear vision to patients of all ages!',
'language': 'en-US', 'source':
'https://www.lumenoptometric.com/orthokeratology/', 'title': 'Orthokeratology |
Lumen Optometric | Sierra Madre, CA | Optometrist'})]
=====

query: What is lumen optometric's address?


c:\Users\ping\MyDrive\py_files\python\py310\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


answr: Lumen Optometric's address is not provided in the context provided.

relevant docs: [Document(page_content='We offer our high-quality vision care
during the following office hours:\nTuesday\n9:45 am – 5:30 pm\nWednesday\n9:45
am – 5:30 pm\nFriday\n9:45 am – 5:30 pm\nSaturday\n9:45 am – 5:30 pm\nWhere to
Park\nWhen you visit the Lumen Optometric office, there are a few convenient
parking options. The map below outlines areas that offer free parking:\nNOTE:
Please avoid parking in the lot adjacent to Happy’s and in the lot directly
behind our building, as those spots are specifically designated for our
neighbors.\nLearn More About Our Exceptional Vision Care Services\nIf you live
or work in our service area, turn to Lumen Optometric for second-to-none eye
care! To schedule an appointment, please call our local team at (626) 921-0199
or contact us online. From routine eye exams to highly specialized vision care,
Lumen Optometric has full-service solutions for a variety of eye care 